# BE : Building Modelling

## Chambeau Maël, Nebon Charles, Bouchet Rémy 

#### Objectives:
- Physical analysis of a house 
- Model a controller for indoor temperature
- Discuss the influence of the controller and of the position of the insulation


## Elements of the thermal network

The thermal resistances for conduction are of the form:
$$R_{cd} = \frac{w}{\lambda S}$$
where:

- $w$ is the width of the material, m;
- $\lambda$ - thermal conductvity, W/m K;
- $S$ - surface area of the wall, m²

The thermals resistance for convection are of the form:
$$R_{cv} = \frac{1}{h S}$$
where:
- $h$ is the convection coefficient, W/m² K;
- $S$ - surface area of the wall, m².

The thermal capacities of the wall are of the form:
$$C_{wall}= \rho_{wall} c_{p, wall} w_{wall} S_{wall}$$

The thermal capacity of the air is:
$$C_{air} = \rho_{air} c_{air} V_{air}$$

The radiative short wave (i.e. solar) heat flow rate on each surface is:
$$ \Phi = S G $$
where:
- $\Phi = \begin{bmatrix}
\Phi_1\\ 
\Phi_2\\ 
...\\ 
\Phi_n
\end{bmatrix}$ - vector of total heat flow rates due to solar radiation [W]; 

- $S =\begin{bmatrix}
S_1 & 0 & ... & 0 \\ 
0 & S_2 & ... & 0 \\ 
... & ... & ... & ...\\ 
0 & 0 & ... & S_n 
\end{bmatrix}$ - matrix of surface areas of walls $i$ [m²].


## Physical analysis and mathematical modelling

![cube](maison.png)
> Simple studio with a bathroom and a living room, including differents electronic components

Let’s consider a cubic building with an HVAC systems controolled by a P-controller.

Here is our choice of sizing for the exterior wall :
![cube](murint.png)

For the window we consider a 100% glass window with e=20mm and the interior wall is a 100% concrete wall with e=160mm.



In [1]:
# P-controler gain
Kp = 1e4            # almost perfect controller Kp -> ∞
Kp = 1e-3           # no controller Kp -> 0

The dimension and surface areas of the building are:

In [2]:
Va = (1.5*1.85+2.56*3.62)*2.5          # m³ volume of air
ACH = 1             # air changes per hour
Va_dot = ACH * Va / 3600    # m³/s air infiltration

#propriétés air
d_air=.2                      # kg/m³
c_air=1000               # J/kg.K

#propriétés bois
d_bois=1400
c_bois=2050
cond_bois=0.065
e_porte=0.05
S_porte=2
#propriétés concrete
d_c=2300
c_c=880
cond_c=1.4
e_c=0.16
S_c=6.590*2.5  # m² surface of concrete & insulation 

#propriété Insulation
d_i=25
c_i=1030
cond_i=0.03
e_i=0.08
S_i=S_c

#propriété placo
d_pla=825
c_pla=1225
cond_pla=0.3
e_pla=0.01
S_pla=S_c

#propriété verre
d_v=2500
c_v=750
cond_v=1.4
e_v=0.02
S_v=3.620*2.5  # m² surface of the window 


ε_wLW = 0.9     # long wave wall emmisivity (concrete)
α_wSW = 0.2     # absortivity white surface
ε_gLW = 0.9     # long wave glass emmisivity (glass pyrex)
τ_gSW = 0.83    # short wave glass transmitance (glass)
α_gSW = 0.1     # short wave glass absortivity

σ = 5.67e-8     # W/m².K⁴ Stefan-Bolzmann constant

Fwg = 1 / 5     # view factor wall - glass

Tm = 20 + 273   # mean temp for radiative exchange

GLW1 = ε_wLW / (1 - ε_wLW) * S_c * 4 * σ * Tm**3
GLW2 = Fwg * S_c * 4 * σ * Tm**3
GLW3 = ε_gLW / (1 - ε_gLW) * S_c* 4 * σ * Tm**3
# long-wave exg. wall-glass
GLW = 1 / (1 / GLW1 + 1 / GLW2 + 1 / GLW3)

Gv = Va_dot * d_air * c_air

g_c =  cond_c/ e_c * S_c
g_pla =  cond_pla/ e_pla * S_pla
g_i =  cond_i/ e_i * S_i
g_v =  cond_v/ e_v * S_v
g_bois =  cond_bois/ e_porte * S_porte

![cube](diag.png)
![cube](leg.png)

The thermal circuit is discribed by the diferential-algebraic set of equations:

$$C \dot{\theta} = -(A^T G A) \theta + A^T G b + f$$
$$q = G (-A \theta + b)$$

where:
- **A** incidence matrix: shows how the temperature nodes are conected by branches of heat flow. It consists of $n_q$ rows and $n_{\theta}$ columns, where $n_q$ is the number of flow branches and $n_{\theta}$  is the number of temperature nodes. If flow *m* enters  into the node *n*, then the element (*m, n*) is 1; if flow *m* exits from the node *n*, then the element (*m, n*) is -1; if flow *m* is not conected to node *n*, then the element (*m, n*) is 0.

- **G** conductance matrix: diagonal matrix containing the conductances. Its size is It consists of $n_q \times n_q$,  where $n_q$ is the number of flow branches.

- **b** temperature source vector: if there is no temperature source on the branch *m*, then $b_m = 0$.

- **C** capacity matrix: diagonal matrix containing the capacities. If there is no capacity in the node *n*, then $C_{n, n} = 0$.

- **f** heat flow source vector: if there is no heat flow source in the node *n*, then $f_n = 0$.


**Incidence matrix**

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import dm4bem
A = np.zeros([14, 12])
A[0, 0] = 1
A[1, 0], A[1, 1] = -1, 1
A[2, 1], A[2, 2] = -1, 1
A[3, 2], A[3, 3] = -1, 1
A[4, 3], A[4, 4] = -1, 1
A[5, 4], A[5, 5] = -1, 1
A[6, 5], A[6, 6] = -1, 1
A[7, 6], A[7, 7] = -1, 1
A[8, 7], A[8, 8] = 1, -1
A[9, 8] = 1
A[10, 7] = 1
A[10, 9], A[11, 9] = -1, 1
A[13, 7], A[11, 10] = 1, -1
A[12, 10], A[12, 11] = 1, -1
A[13, 11] =- 1
print(A)

ModuleNotFoundError: No module named 'numpy'

**Capacity matrix**

In [ ]:
C=np.zeros([12])
C[1]=d_c*c_c*e_c*S_c
C[3]=d_i*c_i*e_i*S_i
C[5]=d_pla*c_pla*e_pla*S_pla
C[8]=d_v*c_v*e_v*S_v
C[10]=d_c*c_c*e_c*S_c
Cdiag=np.diag(C)
print(Cdiag)

**Conductance matrix**

In [ ]:
G=np.zeros([14])

G[1]=25*S_c
G[0]=g_c/2
G[2]=g_c/2
G[3]=g_i/2
G[4]=g_i/2
G[5]=g_pla/2
G[6]=g_pla/2

G[8]=g_v
G[9]=25*S_v
G[10]=g_c/2
G[11]=g_c/2
G[12]=8*(2.02*2.5+1.85*2.50)
G[13]=g_bois
G[7]=GLW+Gv
Gdiag=np.diag(G)